# STAC API Search

A STAC API is the dynamic version of a SpatioTemporal Asset Catalog. Many publishers provide STAC API endpoints that can be queried for specific items.

[STACIndex](https://stacindex.org/) lists all available STAC APIs.

## Setup and Data Download

The following blocks of code will install the required packages and download the datasets to your Colab environment.

In [1]:
  if 'google.colab' in str(get_ipython()):
        !pip install pystac-client --quiet
        !apt install libspatialindex-dev -qq
        !pip install fiona shapely pyproj rtree --quiet
        !pip install geopandas --quiet
        !pip install folium --quiet
        !pip install mapclassify --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.9/182.9 kB 2.1 MB/s eta 0:00:00
The following additional packages will be installed:
  libspatialindex-c6 libspatialindex6
The following NEW packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 319 kB of archives.
After this operation, 1,416 kB of additional disk space will be used.
Selecting previously unselected package libspatialindex6:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libspatialindex6_1.9.3-2_amd64.deb ...
Unpacking libspatialindex6:amd64 (1.9.3-2) ...
Selecting previously unselected package libspatialindex-c6:amd64.
Preparing to unpack .../libspatialindex-c6_1.9.3-2_amd64.deb ...
Unpacking libspatialindex-c6:amd64 (1.9.3-2) ...
Selecting previously unselected package libspatialindex-dev:amd64.
Preparing to unpack .../libspatialindex-dev_1.9.3-2_amd64.deb .

In [12]:
import json
import geopandas as gpd
from shapely.geometry import mapping
import pandas as pd
from pystac_client import Client
import os
import folium
from folium import Figure

In [3]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [4]:
def download(url):
    filename = os.path.join(data_folder, os.path.basename(url))
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/spatialthoughts/python-tutorials/raw/main/data/' +
         'bangalore.geojson')

Downloaded data/bangalore.geojson


## Procedure

Let's open the [USGS Landsat Collection 2 API](https://stacindex.org/catalogs/usgs-landsat-collection-2-api#/).

In [5]:
catalog = Client.open('https://landsatlook.usgs.gov/stac-server')

In [6]:
catalog.extra_fields['conformsTo']

['https://api.stacspec.org/v1.0.0-beta.5/core',
 'https://api.stacspec.org/v1.0.0-beta.5/collections',
 'https://api.stacspec.org/v1.0.0-beta.5/ogcapi-features',
 'https://api.stacspec.org/v1.0.0-beta.5/ogcapi-features#fields',
 'https://api.stacspec.org/v1.0.0-beta.5/ogcapi-features#sort',
 'https://api.stacspec.org/v1.0.0-beta.5/ogcapi-features#query',
 'https://api.stacspec.org/v1.0.0-beta.5/item-search',
 'https://api.stacspec.org/v1.0.0-beta.5/item-search#fields',
 'https://api.stacspec.org/v1.0.0-beta.5/item-search#sort',
 'https://api.stacspec.org/v1.0.0-beta.5/item-search#query',
 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core',
 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30',
 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/geojson']

We get all the collections in the catalog and print their id and description.

In [7]:
collections = catalog.get_all_collections()

datasets = []
for col in collections:
    datasets.append({'collection': col.id, 'description': col.description})
df = pd.DataFrame(datasets)
df

,collection,description
0,landsat-c2l2-sr,The Landsat Surface Reflectance (SR) product m...
1,landsat-c2l2-st,The Landsat Surface Temperature (ST) product r...
2,landsat-c2ard-st,The Landsat Surface Temperature (ST) product r...
3,landsat-c2l2alb-bt,The Landsat Top of Atmosphere Brightness Tempe...
4,landsat-c2l3-fsca,The Landsat Fractional Snow Covered Area (fSCA...
5,landsat-c2ard-bt,The Landsat Top of Atmosphere Brightness Tempe...
6,landsat-c2l1,The Landsat Level-1 product is a top of atmosp...
7,landsat-c2l3-ba,The Landsat Burned Area (BA) contains two acqu...
8,landsat-c2l2alb-st,The Landsat Surface Temperature (ST) product r...
9,landsat-c2ard-sr,The Landsat Surface Reflectance (SR) product m...


In [8]:
aoi_file = 'bangalore.geojson'
aoi_filepath = os.path.join(data_folder, aoi_file)
aoi = gpd.read_file(aoi_filepath)

In [9]:
geometry = aoi.unary_union
geometry_geojson = json.dumps(mapping(geometry))

In [13]:
time_range = "2022-05-01/2022-05-31"

search = catalog.search(
    collections=["landsat-c2l2-sr"],
    intersects=geometry_geojson,
    datetime=time_range,
    query={"eo:cloud_cover": {"lt": 30}},
)
items = search.item_collection()
len(items)

3

In [14]:
items_df = gpd.GeoDataFrame.from_features(items.to_dict(), crs='EPSG:4326')
items_df

,geometry,datetime,eo:cloud_cover,view:sun_azimuth,view:sun_elevation,platform,instruments,view:off_nadir,landsat:cloud_cover_land,landsat:wrs_type,...,accuracy:geometric_x_stddev,accuracy:geometric_y_stddev,accuracy:geometric_rmse,proj:epsg,proj:shape,proj:transform,card4l:specification,card4l:specification_version,created,updated
0,"POLYGON ((76.62028 14.05441, 76.24424 12.31975...",2022-05-24T05:10:48.582484Z,24.20,68.208434,65.588320,LANDSAT_8,"[OLI, TIRS]",0,24.20,2,...,6.004,6.374,8.756,32643,"[7711, 7551]","[30, 0, 633285, 0, -30, 1554915]",SR,5.0,2022-07-01T14:40:27.532Z,2022-07-01T14:40:27.532Z
1,"POLYGON ((76.60833 14.05395, 76.23238 12.31953...",2022-05-08T05:10:44.725871Z,25.09,77.070035,66.309657,LANDSAT_8,"[OLI, TIRS]",0,25.09,2,...,4.510,5.178,6.867,32643,"[7711, 7551]","[30, 0, 632085, 0, -30, 1554915]",SR,5.0,2022-07-01T14:37:02.864Z,2022-07-01T14:37:02.864Z
2,"POLYGON ((76.29250 13.95550, 75.94995 12.33667...",2022-05-08T03:46:31.632068Z,8.00,78.790871,45.955875,LANDSAT_7,[ETM],0,8.00,2,...,3.045,3.435,4.590,32643,"[6931, 7981]","[30, 0, 596685, 0, -30, 1544415]",SR,5.0,2022-09-01T01:41:16.956Z,2022-09-01T01:41:16.956Z


In [15]:
fig = Figure(width=800, height=400)
m = folium.Map()
bounds = items_df.total_bounds
m.fit_bounds([[bounds[1],bounds[0]], [bounds[3],bounds[2]]])


items_df.explore(m=m, color='black', style_kwds={'fillOpacity': 0.2, 'weight': 0.5},)
aoi.explore(m=m, color='blue')
fig.add_child(m)
